# **EA044A – PLANEJAMENTO E ANÁLISE DE SISTEMAS DE PRODUÇÃO**
## Projeto T2 - Planejamento Multiperíodo de Produção

###Bryan Wolff  RA: 214095
###Raphael Cury Spiller  RA: 186300



#**Implementação do Solver**

##**Função Objetivo**


**Variáveis de decisão**

Wn,t : número de pneus com nylon a ser produzido na prensa Wheel no mês t

Ln,t : número de pneus com nylon a ser produzido na prensa Legal no mês t

Wf,t : número de pneus com fibra a ser produzido na prensa Wheel no mês t

Lf,t : número de pneus com fibra a ser produzido na prensa Legal no mês t

En,t : número de pneus com nylon a ser estocado no fim do mês t

Ef,t : número de pneus com fibra a ser estocado no fim do mês t

Sabendo que toda a produção é convertida, ou seja, a receita não varia, para chegarmos na função objetivo basta equacionar os custos para que ele possa ser minimizado e assim maximizar o lucro.

Ao observar as fontes de custo, 3 fontes são encontradas:

*   custo de armazenamento ($\$$0.1 por unid.)

*   custo do material + acabmento,embalagem e transporte ($\$$3.33-nylon $\$$4.13-fibra por unid.)

* custo de produção por hora($\$$5 por hora)  

Contudo, como sabemos a quantidade exata que será produzida em 3 meses,  o custo de material, acabamento, etc, é fixo. 

Diante disso, a função objetivo é dada por:

$(5\cdot0.15) Wn,t + (5\cdot0.12)Wf,t +(5\cdot0.16) Ln,t + (5\cdot0.14) Lf,t + 0.1(Ef,t + En,t)$

em função da quantidade de unidade produzidas por periodo e do estoque por periodo.
Portanto o custo total nos 3 meses será de:


$\sum_{t=1}^{3}(0.75) Wn,t + (0.6)Wf,t +(0.8) Ln,t + (0.7) Lf,t + 0.1(Ef,t + En,t)^*$

*Os estoques em t=3 devem ser iguais a zero.



##**Restrições**

Em relação ao tempo de produção temos que:

período 1

$0.15Wn,1+0.12Wf,1 \leq  700;$

$0,16Ln,1+0,14Lf,1 \leq 1500;$

período 2

$0,15Wn,2+0,12Wf,2 \leq 300;$

$0,16Ln,2+0,14Lf,2 \leq 400;$

período 3

$0,15Wn,3+0,12Wf,3 \leq 1000;$

$0,16Ln,3+0,14Lf,3 \leq 300;$



Em relação à produção excedente estocada:

$En,1 = (Wn,1+Ln,1)-4000;  (Wn,1+Ln,1)$  não pode ser menor que 4000 
$Ef,1 = (Wf,1+Lf,1)-1000; (Wf,1+Lf,1)$  não pode ser menor que 1000 

$En,2 = (Wn,2+Ln,2+En,1)-8000; $
$Ef,2 = (Wf,2+Lf,2+Ef,1)-5000;$ 

sem estoques finais logo, $En,3=Ef,3=0$;

$(Wn,3+Ln,3+En,2)=3000; $
$(Wf,3+Lf,3+Ef,2)=5000; $

##**Solver**
Utilizaremos o solver Or-Tools

###Dependencias

In [ ]:
!pip install --user --upgrade pip setuptools wheel six ortools

In [ ]:
from __future__ import print_function
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

### Declarando Variáveis Utilizadas

In [ ]:
#prensa Wheel
Wn, Wf = [0,0,0], [0,0,0]

#prensa Legal
Ln, Lf = [0,0,0], [0,0,0]

#Estoque
En, Ef = [0,0],[0,0]

#vamos declarar as variaveis a partir do atributo IntVar (Queremos números inteiros positivos)

#WNT : número de pneus com nylon a ser produzido na prensa Wheel no mês T
Wn[0] = solver.IntVar(0, solver.Infinity(), 'WN1')
Wn[1] = solver.IntVar(0, solver.Infinity(), 'WN2')
Wn[2] = solver.IntVar(0, solver.Infinity(), 'WN3')

#WFT : número de pneus com fibra a ser produzido na prensa Wheel no mês T
Wf[0] = solver.IntVar(0, solver.Infinity(), 'WF1')
Wf[1] = solver.IntVar(0, solver.Infinity(), 'WF2')
Wf[2] = solver.IntVar(0, solver.Infinity(), 'WF3')

#LNT : número de pneus com nylon a ser produzido na prensa Legal no mês T
Ln[0] = solver.IntVar(0, solver.Infinity(), 'LN1')
Ln[1] = solver.IntVar(0, solver.Infinity(), 'LN2')
Ln[2] = solver.IntVar(0, solver.Infinity(), 'LN3')

#LFT : número de pneus com fibra a ser produzido na prensa Legal no mês T
Lf[0] = solver.IntVar(0, solver.Infinity(), 'LF1')
Lf[1] = solver.IntVar(0, solver.Infinity(), 'LF2')
Lf[2] = solver.IntVar(0, solver.Infinity(), 'LF3')

#ENT : número de pneus com nylon a ser estocado no fim do mês T
En[0] = solver.IntVar(0, solver.Infinity(), 'EN1')
En[1] = solver.IntVar(0, solver.Infinity(), 'EN2')

#EFT : número de pneus com fibra a ser estocado no fim do mês T
Ef[0] = solver.IntVar(0, solver.Infinity(), 'EF1')
Ef[1] = solver.IntVar(0, solver.Infinity(), 'EF2')

### Restrições

In [ ]:
#Horas (CAPACIDADE) --------------

#0.15WN1+ 0.12WF1≤ 700
ct1 = solver.Constraint(0,700,'ct1')
ct1.SetCoefficient(Wn[0],0.15)
ct1.SetCoefficient(Wf[0],0.12)

#0.15WN2 + 0.12WG2 ≤ 300
ct2 = solver.Constraint(0,300,'ct2')
ct2.SetCoefficient(Wn[1],0.15)
ct2.SetCoefficient(Wf[1],0.12)

#0.15WN3 + 0.12WF3 ≤ 1000
ct3 = solver.Constraint(0,1000,'ct3')
#ct3 = solver.Constraint(0,1172,'ct3') #questao 4
ct3.SetCoefficient(Wn[2],0.15)
ct3.SetCoefficient(Wf[2],0.12)

#0.16LN1 + 0.14LF1 ≤ 1500
ct4 = solver.Constraint(0,1500,'ct4')
ct4.SetCoefficient(Ln[0],0.16)
ct4.SetCoefficient(Lf[0],0.14)

#0.16LN2 + 0.14LF2 ≤ 400
ct5 = solver.Constraint(0,400,'ct5')
ct5.SetCoefficient(Ln[1],0.16)
ct5.SetCoefficient(Lf[1],0.14)

#0.16LN3 + 0.14LF3 ≤ 300
ct6 = solver.Constraint(0,300,'ct6')
ct6.SetCoefficient(Ln[2],0.16)
ct6.SetCoefficient(Lf[2],0.14)


#Demanda --------------

#JUNHO ---
#WN1 + LN1 - EN1 = 4000
ct7 = solver.Constraint(4000,4000,'ct7')
ct7.SetCoefficient(Wn[0],1)
ct7.SetCoefficient(Ln[0],1)
ct7.SetCoefficient(En[0],-1)

#WF1 + LF1 - EF1 = 1000
ct8 = solver.Constraint(1000,1000,'ct8')
ct8.SetCoefficient(Wf[0],1)
ct8.SetCoefficient(Lf[0],1)
ct8.SetCoefficient(Ef[0],-1)

#JULHO ---
#EN1 + WN2 + LN2 - EN2 = 8000
ct9 = solver.Constraint(8000,8000,'ct9')
ct9.SetCoefficient(En[0],1)
ct9.SetCoefficient(Wn[1],1)
ct9.SetCoefficient(Ln[1],1)
ct9.SetCoefficient(En[1],-1)

#EF1 + WF2 + LF2 - EF2 = 5000
ct10 = solver.Constraint(5000,5000,'ct10')
ct10.SetCoefficient(Ef[0],1)
ct10.SetCoefficient(Wf[1],1)
ct10.SetCoefficient(Lf[1],1)
ct10.SetCoefficient(Ef[1],-1)

#AGOSTO ---
#EN2 + WN3 + LN3 = 3000
ct11 = solver.Constraint(3000,3000,'ct11')
ct11.SetCoefficient(En[1],1)
ct11.SetCoefficient(Wn[2],1)
ct11.SetCoefficient(Ln[2],1)

#EF2 + WF3 + LF3 = 5000
ct12 = solver.Constraint(5000,5000,'ct12')
ct12.SetCoefficient(Ef[1],1)
ct12.SetCoefficient(Wf[2],1)
ct12.SetCoefficient(Lf[2],1)

### Função Objetivo

In [ ]:
objective = solver.Objective()

#0.75WNT + 0.8LNT + 0.6WFT + 0.7LFT + 0.1ENT + 0.1EFT

for i in range(0,3):
  objective.SetCoefficient(Wn[i],0.75)
  objective.SetCoefficient(Ln[i],0.8)
  objective.SetCoefficient(Wf[i],0.6)
  objective.SetCoefficient(Lf[i],0.7)

for i in range(0,2):
  objective.SetCoefficient(En[i],0.1)
  objective.SetCoefficient(Ef[i],0.1)

objective.SetMinimization()
solver.Solve()


0

#**Questão 1** 
## *Qual é a produção mensal que atende a demanda exatamente com menor custo?*



### Solução

In [ ]:
for i in range(0,3):
  if i == 0: print("Mês de Junho")
  elif i == 1: print("Mês de Julho")
  else: print("Mês de Agosto")
  print("n° pneus de NYLON produzidos pela WHEEL",Wn[i].solution_value())
  print("n° pneus de FIBRA produzidos pela WHEEL",Wf[i].solution_value())
  print("n° pneus de NYLON produzidos pela LEGAL",Ln[i].solution_value())
  print("n° pneus de FIBRA produzidos pela LEGAL",Lf[i].solution_value())
  if i<2:
    print("Estoque de pneus de NYLON", En[i].solution_value())
    print("Estoque de pneus de FIBRA", Ef[i].solution_value())
  else: 
    print("Estoque de pneus de NYLON", 0.0)
    print("Estoque de pneus de FIBRA", 0.0)
  print('\n')

print("Custo total: ", objective.Value())

Mês de Junho
n° pneus de NYLON produzidos pela WHEEL 1866.0
n° pneus de FIBRA produzidos pela WHEEL 3500.0
n° pneus de NYLON produzidos pela LEGAL 7634.0
n° pneus de FIBRA produzidos pela LEGAL 0.0
Estoque de pneus de NYLON 5500.0
Estoque de pneus de FIBRA 2500.0


Mês de Julho
n° pneus de NYLON produzidos pela WHEEL -0.0
n° pneus de FIBRA produzidos pela WHEEL 2500.0
n° pneus de NYLON produzidos pela LEGAL 2500.0
n° pneus de FIBRA produzidos pela LEGAL 0.0
Estoque de pneus de NYLON 0.0
Estoque de pneus de FIBRA 0.0


Mês de Agosto
n° pneus de NYLON produzidos pela WHEEL 2666.0
n° pneus de FIBRA produzidos pela WHEEL 5000.0
n° pneus de NYLON produzidos pela LEGAL 334.0
n° pneus de FIBRA produzidos pela LEGAL 0.0
Estoque de pneus de NYLON 0.0
Estoque de pneus de FIBRA 0.0


Custo total:  38330.1


#**Questão 2**
## *Qual a receita, o custo e o lucro para o período junho-julho-agosto?*

###**Receita:**
 - Nylon = nº pneus nylon produzidos * preço do nylon = 15000•7 = 105000
 - Fibra = nº Pneus fibra produzidos * preço do fibra = 11000•9 = 99000
 - Receita = 105000 + 99000 = **$204000**


###**Custos:**
 - Matéria prima dos pneus de nylon: 3.10*15000 = 46500
 - Matéria prima dos pneus de fibra: 3.90*11000 = 42900
 - Custo de produção: 19173.4
 - Custos de Acabamento, embalagem e transporte = 0.23*(11000 + 15000) = 5980
 - Total = 46500 + 42900 + 19173.4 + 5980 = **$114553.40**


###**Lucro:**
 - Lucro = Receita - Custos = 204000 - 114553,4 = **$89446.60**

#**Questão 3**
## *Quando seria apropriado fazer a manutenção preventiva das prensas?*


O calculo das folgas é feito a partir da quantidade de unidades produzidas por periodo(para que obtenhamos maior lucro). Sabemos o tempo que cada maquina leva em horas para produzir cada tipo de pneu, portanto:

**Junho:**
 - **Horas de produção previstas**:(W-700,L-1500)
 - WHEEL: $1866\cdot0.15+3500\cdot0.12 =699.9h$
 - LEGAL: $7634\cdot0.16+0\cdot0.14 =1221.44h$

**Julho:**
 - **Horas de produção previstas**:(W-300,L-400)
 - WHEEL: $0\cdot0.15+2500\cdot0.12 =300h$
 - LEGAL: $2500\cdot0.16+0\cdot0.14 =400h$

**Agosto:**
 - **Horas de produção previstas**:(W-1000,L-300)
 - WHEEL: $2666\cdot0.15+5000\cdot0.12 =999.9h$
 - LEGAL: $334\cdot0.16+0\cdot0.14 =53.44h$

**FOLGA DAS MAQUINAS**:

**Junho:**
 - WHEEL: 0 horas
 - LEGAL: 1500-1221.44 = 279horas

**Julho:**
 - WHEEL:0 horas
 - LEGAL: 0 horas

**Agosto:**
 - WHEEL: 0 horas
 - LEGAL: 300-53 = 247 horas

In [ ]:
#código para calculo
print("Folgas de cada máquina:")
print("\n")

print("-> Mês de Junho")
print("Folga (h) da prensa Wheel:",round(-0.15*Wn[0].solution_value() - 0.12*Wf[0].solution_value() + 700))
print("Folga (h) da prensa Legal:",round(-0.16*Ln[0].solution_value() - 0.14*Lf[0].solution_value() + 1500))
print("\n")

print("-> Mês de Julho")
print("Folga (h) da prensa Wheel:",round(-0.15*Wn[1].solution_value() - 0.12*Wf[1].solution_value() + 300))
print("Folga (h) da prensa Legal:",round(-0.16*Ln[1].solution_value() - 0.14*Lf[1].solution_value() + 400))
print("\n")

print("-> Mês de Agosto")
print("Folga (h) da prensa Wheel:",round(-0.15*Wn[2].solution_value() - 0.12*Wf[2].solution_value() + 1000))
print("Folga (h) da prensa Legal:",round(-0.16*Ln[2].solution_value() - 0.14*Lf[2].solution_value() + 300))
print("\n")

Folgas de cada máquina:


-> Mês de Junho
Folga (h) da prensa Wheel: 0
Folga (h) da prensa Legal: 279


-> Mês de Julho
Folga (h) da prensa Wheel: 0
Folga (h) da prensa Legal: 0


-> Mês de Agosto
Folga (h) da prensa Wheel: 0
Folga (h) da prensa Legal: 247




A prensa Legal possuí folgas de 279h e 247h respectivamente nos meses de Junho e Agosto, sendo esses os momentos apropriado para realizar a manutenção na maquina. Porém, a prensa Wheel não possuí nenhuma folga em nenhum desses meses, ou seja, a manutenção impactará na produção.

#**Questão 4**
##*Uma nova prensa do tipo Wheel está para chegar em setembro. Contudo, é possível antecipar sua chegada para 2 de agosto desde que a Peneu pague $200 ao seu fornecedor. O número de horas de produção adicionais ganho com a chegada da nova prensa é 172. A Peneu deve antecipar a chegada da nova prensa?*


Executando novamente o solver mudando a restrição $0,15Wn,3+0,12Wf,3 \leq 1000$ para $0,15Wn,3+0,12Wf,3 \leq 1172;$ obtemos o seguinte resultado

Mês de Junho
 - n° pneus de NYLON produzidos pela WHEEL 1866.0
 - n° pneus de FIBRA produzidos pela WHEEL 3500.0
 - n° pneus de NYLON produzidos pela LEGAL 7634.0
 - n° pneus de FIBRA produzidos pela LEGAL 0.0
 - Estoque de pneus de NYLON 5500.0
 - Estoque de pneus de FIBRA 2500.0


Mês de Julho
 - n° pneus de NYLON produzidos pela WHEEL -0.0
 - n° pneus de FIBRA produzidos pela WHEEL 2500.0
 - n° pneus de NYLON produzidos pela LEGAL 2500.0
 - n° pneus de FIBRA produzidos pela LEGAL 0.0
 - Estoque de pneus de NYLON 0.0
 - Estoque de pneus de FIBRA 0.0


Mês de Agosto
 - n° pneus de NYLON produzidos pela WHEEL 3000.0
 - n° pneus de FIBRA produzidos pela WHEEL 5000.0
 - n° pneus de NYLON produzidos pela LEGAL -0.0
 - n° pneus de FIBRA produzidos pela LEGAL -0.0
 - Estoque de pneus de NYLON 0.0
 - Estoque de pneus de FIBRA 0.0


Custo total:  19156.7

Agora, vamos calcular a nova receita, custo e o lucro para o período

###**Receita:**
 - Nylon = nº pneus nylon produzidos * preço do nylon = 15000•7 = 105000
 - Fibra = nº Pneus fibra produzidos * preço do fibra = 11000•9 = 99000
 - Receita = 105000 + 99000 = **$204000**


###**Custos:**
 - Matéria prima dos pneus de nylon: 3.10*15000 = 46500
 - Matéria prima dos pneus de fibra: 3.90*11000 = 42900
 - Custo de produção: 19156.4
 - Custo de adiantamente na prensa Peneu: 200
 - Custos de Acabamento, embalagem e transporte = 0.23*(11000 + 15000) = 5980
 - Total = 46500 + 42900 + 19156.4 + 5980 + 200 = **$114736.40**


###**Lucro:**
 - Lucro = Receita - Custos = 204000 - 114736.40 = **$89263.60**

Ao obter o novo lucro e custo com o adiantamento da nova prensa Wheel e, ao comparar com o valor obtido anteriormente na questão 1, é notável um certo prejuízo devido ao aumento do custo e com o decréscimo do lucro. Portanto, não é recomendado antecipar a chegada da nova prensa.

#**Questão 5**
##*Resolver o modelo usando o Excel, Or-Tools, ou o solver de sua preferência*

Foi utilizado o solver Or-Tools para a resolução do problema, e sua implementação foi feita ao decorrer do relatório